# Armoria API

In [1]:
# auto-load when code changes outside
%load_ext autoreload
%autoreload 2
%load_ext pyinstrument

import sys
sys.path.insert(0,'..')


import os
from src.armoria_api import ArmoriaAPIPayload, ArmoriaAPIWrapper
from src.armoria_api_generator_helper import ArmoriaAPIGeneratorHelper

from src.caption import Caption


# Examples of using the ArmoriaAPIWrapper 

In [2]:
coa_lion = {"t1":"or",
       "shield":"heater",
       "charges":[{"charge":"lionRampant","t":"gules","p":"e","size":1.5}],
       "ordinaries":[{"ordinary":"bordure","t":"azure"}],
      }

api = ArmoriaAPIWrapper(size = 500,format="png", coa=coa_lion)
api.show_image()


In [3]:

coa_cross = {"t1":"argent",
       "shield":"heater",
       "charges":[{"charge":"crossMoline",
                   "t":"azure",
                   "p":"e",
                   "size":1.5}]
      }

api = ArmoriaAPIWrapper(size = 500,format="png", coa=coa_cross)
api.show_image()

In [4]:
payload = {'t1': 'azure', 'shield': 'heater', 
             'charges': [{'charge': 'lionRampant', 't': 'argent', 'p': 'k', 'size': '0.7'},
                        {'charge': 'eagle', 't': 'or', 'p': 'n', 'size': '0.7'}],
             'ordinaries': []}
api = ArmoriaAPIWrapper(size = 500,format="png", coa=payload)
api.show_image()


In [5]:
payload = {'t1': 'or', 'shield': 'heater', 
             'charges': [{'charge': 'lionRampant', 't': 'or', 'p': 'abc', 'size': '0.3'},
                         {'charge': 'eagle', 't': 'azure', 'p': 'dfgz', 'size': '0.3'},
                         {'charge': 'crossHummetty', 't': 'argent', 'p': 'i', 'size': '0.3'}],
             'ordinaries': [{"ordinary":"bordure", "t":"azure"}]}

api = ArmoriaAPIWrapper(size = 500,format="png", coa=payload)
api.show_image()


# Generate Permutations

In [3]:
import itertools
import src.alphabet as alphabet
from src.armoria_api import COLORS_MAP, SINGLE_LION_MODIFIERS_MAP, PLURAL_LION_MODIFIERS_MAP, CROSS_MODIFIERS_MAP, \
SINGLE_EAGLE_MODIFIERS_MAP, PLURAL_EAGLE_MODIFIERS_MAP, \
POSITIONS, SIZES, NUMBERS, NUMBERS_MULTI, SINGLE_POSITION

BORDER_MOD = ['& border', '']

## Single object

In [4]:

# lion, modifiers and colors
permutations1 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list( SINGLE_LION_MODIFIERS_MAP.keys()),BORDER_MOD)))

# cross, modifiers and colors
permutations2 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list( CROSS_MODIFIERS_MAP.keys()),BORDER_MOD)))

# eagle, modifiers and colors
permutations3 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list( SINGLE_EAGLE_MODIFIERS_MAP.keys()),BORDER_MOD)))

possible_single_permutations = permutations1 + permutations2 + permutations3

print('Total number of permutations:', len(possible_single_permutations))

Total number of permutations: 720


## Plural Object with Number

In [8]:
# lion, modifiers and colors
permutations1 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list(NUMBERS), list( PLURAL_LION_MODIFIERS_MAP.keys()),BORDER_MOD)))

# eagle, modifiers and colors
permutations2 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list(NUMBERS), list( PLURAL_EAGLE_MODIFIERS_MAP.keys()),BORDER_MOD)))

possible_pl_permutations = permutations1 + permutations2

print('Total number of plural permutations:', len(possible_pl_permutations))


Total number of plural permutations: 4320


## Multi Objects - single

In [9]:

# lion & eagle
permutations1 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()),list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list( SINGLE_LION_MODIFIERS_MAP.keys()), list( SINGLE_EAGLE_MODIFIERS_MAP.keys()),BORDER_MOD)))

# lion & cross
permutations2 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()),list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list( SINGLE_LION_MODIFIERS_MAP.keys()), list( CROSS_MODIFIERS_MAP.keys()),BORDER_MOD)))

# eagle & cross
permutations3 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()),list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list( SINGLE_EAGLE_MODIFIERS_MAP.keys()), list( CROSS_MODIFIERS_MAP.keys()),BORDER_MOD)))

# lion & cross & eagle
permutations4 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()),list(COLORS_MAP.keys()),list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list( SINGLE_LION_MODIFIERS_MAP.keys()), list( SINGLE_EAGLE_MODIFIERS_MAP.keys()), list( CROSS_MODIFIERS_MAP.keys()),BORDER_MOD)))

possible_multi_single_permutations = permutations1 + permutations2 + permutations3 + permutations4

print('Total number of permutations:', len(possible_multi_single_permutations))


Total number of permutations: 91152


## Multi Objects - plural

In [10]:

# lion & eagle
permutations1 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()),list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list(NUMBERS_MULTI),list( PLURAL_LION_MODIFIERS_MAP.keys()), list(NUMBERS_MULTI), list( PLURAL_EAGLE_MODIFIERS_MAP.keys()),BORDER_MOD)))

# lion & cross
permutations2 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()),list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list(NUMBERS_MULTI),list( PLURAL_LION_MODIFIERS_MAP.keys()), list( CROSS_MODIFIERS_MAP.keys()),BORDER_MOD)))

# eagle & cross
permutations3 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()),list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list(NUMBERS_MULTI),list( PLURAL_EAGLE_MODIFIERS_MAP.keys()), list( CROSS_MODIFIERS_MAP.keys()),BORDER_MOD)))

# lion & cross & eagle
permutations4 = list(dict.fromkeys(itertools.product(list(COLORS_MAP.keys()),list(COLORS_MAP.keys()),list(COLORS_MAP.keys()), list(COLORS_MAP.keys()), list(NUMBERS_MULTI), list( PLURAL_LION_MODIFIERS_MAP.keys()), list(NUMBERS_MULTI),list( PLURAL_EAGLE_MODIFIERS_MAP.keys()), list( CROSS_MODIFIERS_MAP.keys()),BORDER_MOD)))

possible_multi_plural_permutations = permutations1 + permutations2 + permutations3 + permutations4

print('Total number of plural permutations:', len(possible_multi_plural_permutations))


Total number of plural permutations: 1081728


## Total number of permutations before adding border (588960)

## Total number of permutations with/out borders (1177920)
## __NOTE__: Not possible to generate this amount of images with the Armoria API

In [11]:
total_possible_permutations = possible_single_permutations + possible_pl_permutations + \
                            possible_multi_single_permutations  + possible_multi_plural_permutations
len(total_possible_permutations)

1177920

In [12]:
total_possible_permutations[0]

('A', 'A', 'lion', '& border')

# Take only 2000 random sample from the permutation list

In [20]:
import random

subset_permutations = random.sample(total_possible_permutations, 2000)
len(subset_permutations)

2000

In [21]:
subset_permutations[:10]

[('V',
  'O',
  'S',
  'O',
  '4',
  'lions',
  '3',
  'eagles doubleheaded',
  'cross patonce',
  '& border'),
 ('O', 'S', 'O', 'B', '5', 'lions passt', '5', 'eagles', 'cross', ''),
 ('V',
  'G',
  'O',
  'B',
  '4',
  'lions passt',
  '2',
  'eagles doubleheaded',
  'cross',
  '& border'),
 ('S',
  'A',
  'S',
  'A',
  '5',
  'lions passt guard',
  '5',
  'eagles doubleheaded',
  'cross',
  '& border'),
 ('V', 'S', 'B', '3', 'lions', '5', 'eagles', ''),
 ('V',
  'G',
  'B',
  'G',
  '2',
  'lions rampant',
  '5',
  'eagles doubleheaded',
  'cross',
  ''),
 ('O', 'A', 'A', 'V', '5', 'lions passt', '3', 'eagles', 'cross', ''),
 ('G',
  'G',
  'A',
  'V',
  '5',
  'lions rampant',
  '3',
  'eagles',
  'cross',
  '& border'),
 ('B',
  'V',
  'S',
  'V',
  '4',
  'lions passt guard',
  '3',
  'eagles doubleheaded',
  'cross',
  ''),
 ('G', 'B', 'B', 'V', '3', 'lions rampant', '3', 'eagles', 'cross moline', '')]

## Total number of single permutations +  position and scale

In [5]:
# FOLDER_NAME = '/home/space/datasets/COA/generated-data-api-subset-random'
FOLDER_NAME = '../generated'
caption_file = FOLDER_NAME + '/' + 'captions.txt'
api_gen_helper = ArmoriaAPIGeneratorHelper(caption_file, FOLDER_NAME, subset_permutations) 

In [6]:
api_gen_helper.creat_caption_file()

In [7]:
api_gen_helper.generate_caption_file()

In [ ]:
api_gen_helper.generate_dataset()